## Merging the pg&e data set with geographic data(city and lat/long coords)

In [1]:
import pandas as pd
import numpy as np
import os
import pprint as pp
import geopandas as gpd


# !pip install geopandas

In [2]:
!ls data

7882666-5bdc46db47d9515269ab12ed6fb2850377fd869e.zip
Bay Area Zip Codes + City Name - Sheet1.csv
CA_Counties
PGE
US-Zip-Codes-from-2013-Government-Data.csv
us-zip-code-latitude-and-longitude
us-zip-code-latitude-and-longitude.csv
us-zip-code-latitude-and-longitude.zip


In [3]:
def info(df):
    print(df.shape)
    print(df.columns)
    display(df.head())

In [51]:
full_cleaned_fn = "cleaned_data/pge-monthly-full-cleaned-by-zip_2013-2020.csv"
full_cleaned_df = pd.read_csv(full_cleaned_fn) ## ZIPCODE neds tot be string ttype to merge with latlong mapping dataframe

info(full_cleaned_df)
type(full_cleaned_df['ZIPCODE'][0])

(154283, 17)
Index(['Unnamed: 0', 'ZIPCODE', 'DATE', 'MONTH', 'YEAR', 'DAY',
       'CUSTOMERCLASS', 'COMBINED', 'TOTALCUSTOMERS', 'TOTALKWH', 'AVERAGEKWH',
       'TOTALTHERMS', 'AVERAGETHERMS', 'Calculated AVGKWH',
       'Calculated AVGTHERMS', 'AVGKWH Diff', 'AVGTHERMS Diff'],
      dtype='object')


,Unnamed: 0,ZIPCODE,DATE,MONTH,YEAR,DAY,CUSTOMERCLASS,COMBINED,TOTALCUSTOMERS,TOTALKWH,AVERAGEKWH,TOTALTHERMS,AVERAGETHERMS,Calculated AVGKWH,Calculated AVGTHERMS,AVGKWH Diff,AVGTHERMS Diff
0,0,93101,2013-01-01,1,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
1,1,93101,2013-02-01,2,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
2,2,93101,2013-03-01,3,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
3,3,93105,2013-01-01,1,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
4,4,93105,2013-02-01,2,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0


numpy.int64

In [5]:
## this is the geographic data in a shapefile
zipcodes_to_latlong_fn = "data/US-Zip-Codes-from-2013-Government-Data.csv"
zipcodes_to_latlong_df = pd.read_csv(zipcodes_to_latlong_fn)

info(zipcodes_to_latlong_df)
type(zipcodes_to_latlong_df['ZIP'][0])

(33144, 3)
Index(['ZIP', 'LAT', 'LNG'], dtype='object')


,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


numpy.int64

In [24]:
zipcodes_to_latlong_df[zipcodes_to_latlong_df['ZIP']== 94080]

,ZIP,LAT,LNG
30982,94080,37.65549,-122.422075


In [10]:
pge_with_zip_coords = pd.merge(left=full_cleaned_df, right=zipcodes_to_latlong_df, left_on="ZIPCODE", right_on='ZIP', how='left')

There are some zip codes in the pg&e dataset that is not in the zip codes to longitude latitude mapping. We need to fill these out.

The process to do this is:
1. find the missing zipcodes
2. add rows to the zipcodes dataframe that includes these zip codes and their corresponding coordinates
3. remerge with the pg&e data

In [16]:
pge_with_zip_coords[pge_with_zip_coords['ZIP'].isna()]['ZIPCODE'].unique()


array([93435, 93737, 93741, 94018, 94562, 95044, 95141, 95229, 95309,
       95419, 95480, 95518, 95613, 95656, 95676, 95913, 95972, 94035,
       96162, 95836, 93944])

In [34]:
missing_zips_coords = {
    93435: {'LAT': 35.5034, 'LNG': -121.0392},
    93737: {'LAT': 36.7513, 'LNG': -119.6441},
    93741: {'LAT': 36.7668, 'LNG': -119.7994},
    94018: {'LAT': 37.5017, 'LNG': -122.4688},
    94562: {'LAT': 38.4373, 'LNG': -122.4030},
    95044: {'LAT': 37.1583, 'LNG': -121.9865},
    95141: {'LAT': 37.1677, 'LNG': -121.7708},
    95229: {'LAT': 38.0900, 'LNG': -120.4700},
    95309: {'LAT': 37.8697, 'LNG': -120.4170},
    95419: {'LAT': 38.4249, 'LNG': -122.9582},
    95480: {'LAT': 38.7445, 'LNG': -123.4776},
    95518: {'LAT': 40.8700, 'LNG': -124.0800},
    95613: {'LAT': 38.7973, 'LNG': -120.8883},
    95656: {'LAT': 38.5516, 'LNG': -120.7307},
    95676: {'LAT': 38.8773, 'LNG': -121.6853},
    95913: {'LAT': 39.6330, 'LNG': -122.2100},
    95972: {'LAT': 39.4539, 'LNG': -121.3141},
    94035: {'LAT': 37.4175, 'LNG': -122.0525},
    96162: {'LAT': 39.3200, 'LNG': -120.3000},
    95836: {'LAT': 38.7143, 'LNG': -121.5405},
    93944: {'LAT': 36.6032, 'LNG': -121.9146}
}
missing_zips_df=pd.DataFrame(missing_zips_coords).T.reset_index().rename({'index':'ZIP'}, axis=1)
missing_zips_df.head()

,ZIP,LAT,LNG
0,93435,35.5034,-121.0392
1,93737,36.7513,-119.6441
2,93741,36.7668,-119.7994
3,94018,37.5017,-122.4688
4,94562,38.4373,-122.4030


In [39]:
zipcodes_to_latlong_all_df = pd.concat([missing_zips_df, zipcodes_to_latlong_df])
info(zipcodes_to_latlong_all_df)

(33165, 3)
Index(['ZIP', 'LAT', 'LNG'], dtype='object')


,ZIP,LAT,LNG
0,93435,35.5034,-121.0392
1,93737,36.7513,-119.6441
2,93741,36.7668,-119.7994
3,94018,37.5017,-122.4688
4,94562,38.4373,-122.4030


In [40]:
pge_with_all_zip_coords = pd.merge(left=full_cleaned_df, right=zipcodes_to_latlong_all_df, left_on="ZIPCODE", right_on='ZIP', how='inner')
info(pge_with_all_zip_coords)


(154283, 20)
Index(['Unnamed: 0', 'ZIPCODE', 'DATE', 'MONTH', 'YEAR', 'DAY',
       'CUSTOMERCLASS', 'COMBINED', 'TOTALCUSTOMERS', 'TOTALKWH', 'AVERAGEKWH',
       'TOTALTHERMS', 'AVERAGETHERMS', 'Calculated AVGKWH',
       'Calculated AVGTHERMS', 'AVGKWH Diff', 'AVGTHERMS Diff', 'ZIP', 'LAT',
       'LNG'],
      dtype='object')


,Unnamed: 0,ZIPCODE,DATE,MONTH,YEAR,DAY,CUSTOMERCLASS,COMBINED,TOTALCUSTOMERS,TOTALKWH,AVERAGEKWH,TOTALTHERMS,AVERAGETHERMS,Calculated AVGKWH,Calculated AVGTHERMS,AVGKWH Diff,AVGTHERMS Diff,ZIP,LAT,LNG
0,0,93101,2013-01-01,1,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0,93101,34.418979,-119.709166
1,1,93101,2013-02-01,2,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0,93101,34.418979,-119.709166
2,2,93101,2013-03-01,3,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0,93101,34.418979,-119.709166
3,4341,93101,2013-04-01,4,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0,93101,34.418979,-119.709166
4,4342,93101,2013-05-01,5,2013,1,Elec- Residential,Y,0,0,0,0,0,0.0,0.0,0.0,0.0,93101,34.418979,-119.709166


In [42]:
## there should be no more zipcodes with missing coordinates in the pg&e dataset
pge_with_all_zip_coords[pge_with_all_zip_coords['ZIP'].isna()]['ZIPCODE'].unique()


array([], dtype=int64)

In [22]:
zipcodes_to_latlong_gdf[zipcodes_to_latlong_gdf['zip']=='93314']

,Unnamed: 0,ZIPCODE,DATE,MONTH,YEAR,DAY,CUSTOMERCLASS,COMBINED,TOTALCUSTOMERS,TOTALKWH,AVERAGEKWH,TOTALTHERMS,AVERAGETHERMS,Calculated AVGKWH,Calculated AVGTHERMS,AVGKWH Diff,AVGTHERMS Diff
144,144,93314,2013-01-01,1,2013,1,Elec- Residential,N,6719,5619273,836,0,0,836.325793,0.000000,0.325793,0.000000
145,145,93314,2013-02-01,2,2013,1,Elec- Residential,N,6870,4612359,671,0,0,671.376856,0.000000,0.376856,0.000000
146,146,93314,2013-03-01,3,2013,1,Elec- Residential,N,7150,4751023,664,0,0,664.478741,0.000000,0.478741,0.000000
2628,2628,93314,2013-01-01,1,2013,1,Gas- Residential,Y,897,0,0,92317,103,0.000000,102.917503,0.000000,-0.082497
2629,2629,93314,2013-02-01,2,2013,1,Gas- Residential,Y,907,0,0,59476,66,0.000000,65.574421,0.000000,-0.425579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150105,150105,93314,2020-11-01,11,2020,1,Elec- Residential,N,9945,4695586,472,0,0,472.155455,0.000000,0.155455,0.000000
150106,150106,93314,2020-12-01,12,2020,1,Elec- Residential,N,9962,6407995,643,0,0,643.243827,0.000000,0.243827,0.000000
152594,152594,93314,2020-10-01,10,2020,1,Gas- Residential,Y,1490,0,0,38468,26,0.000000,25.817450,0.000000,-0.182550
152595,152595,93314,2020-11-01,11,2020,1,Gas- Residential,Y,1502,0,0,68339,45,0.000000,45.498668,0.000000,0.498668


In [44]:
type(pge_with_all_zip_coords)

pandas.core.frame.DataFrame

## Using gpd zips
GPD is geopandas which will allow us to make nice geographical maps

In [45]:
zips_gpd = gpd.read_file('data/us-zip-code-latitude-and-longitude/us-zip-code-latitude-and-longitude.shp')

In [72]:
f = gpd.read_file('cleaned_data/pge-monthly-consumption_2013-2020_with-zipcodes-and-cities.shp')
info(f)

(33611, 21)
Index(['zip', 'city', 'state', 'latitude', 'longitude', 'timezone', 'dst',
       'ZIPCODE', 'MONTH', 'YEAR', 'CUSTOMERCL', 'COMBINED', 'TOTALCUSTO',
       'TOTALKWH', 'AVERAGEKWH', 'TOTALTHM', 'AVERAGETHM', 'TOTALTHERM',
       'AVERAGETHE', 'City_1', 'geometry'],
      dtype='object')


,zip,city,state,latitude,longitude,timezone,dst,ZIPCODE,MONTH,YEAR,...,COMBINED,TOTALCUSTO,TOTALKWH,AVERAGEKWH,TOTALTHM,AVERAGETHM,TOTALTHERM,AVERAGETHE,City_1,geometry
0,94564,Pinole,CA,37.997509,-122.29208,-8.0,1.0,94564,1,2013,...,N,6727,3787580,563,0,0,0,0,Pinole,POINT (-122.29208 37.99751)
1,94564,Pinole,CA,37.997509,-122.29208,-8.0,1.0,94564,2,2013,...,N,6725,3086143,459,0,0,0,0,Pinole,POINT (-122.29208 37.99751)
2,94564,Pinole,CA,37.997509,-122.29208,-8.0,1.0,94564,3,2013,...,N,6715,3091798,460,0,0,0,0,Pinole,POINT (-122.29208 37.99751)
3,94564,Pinole,CA,37.997509,-122.29208,-8.0,1.0,94564,1,2013,...,N,6166,0,0,562494,91,0,0,Pinole,POINT (-122.29208 37.99751)
4,94564,Pinole,CA,37.997509,-122.29208,-8.0,1.0,94564,2,2013,...,N,6172,0,0,388246,63,0,0,Pinole,POINT (-122.29208 37.99751)


In [49]:
zips_gpd['zip'] = zips_gpd['zip'].astype('int')

In [53]:
pge_with_all_zip_coords_gpd = pd.merge(left=zips_gpd, right=full_cleaned_df, left_on="zip", right_on='ZIPCODE', how='right')
info(pge_with_all_zip_coords_gpd)


(154283, 25)
Index(['zip', 'city', 'state', 'latitude', 'longitude', 'timezone', 'dst',
       'geometry', 'Unnamed: 0', 'ZIPCODE', 'DATE', 'MONTH', 'YEAR', 'DAY',
       'CUSTOMERCLASS', 'COMBINED', 'TOTALCUSTOMERS', 'TOTALKWH', 'AVERAGEKWH',
       'TOTALTHERMS', 'AVERAGETHERMS', 'Calculated AVGKWH',
       'Calculated AVGTHERMS', 'AVGKWH Diff', 'AVGTHERMS Diff'],
      dtype='object')


,zip,city,state,latitude,longitude,timezone,dst,geometry,Unnamed: 0,ZIPCODE,...,COMBINED,TOTALCUSTOMERS,TOTALKWH,AVERAGEKWH,TOTALTHERMS,AVERAGETHERMS,Calculated AVGKWH,Calculated AVGTHERMS,AVGKWH Diff,AVGTHERMS Diff
0,93101.0,Santa Barbara,CA,34.420163,-119.7065,-8.0,1.0,POINT (-119.70650 34.42016),0,93101,...,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
1,93101.0,Santa Barbara,CA,34.420163,-119.7065,-8.0,1.0,POINT (-119.70650 34.42016),1,93101,...,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
2,93101.0,Santa Barbara,CA,34.420163,-119.7065,-8.0,1.0,POINT (-119.70650 34.42016),2,93101,...,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
3,93101.0,Santa Barbara,CA,34.420163,-119.7065,-8.0,1.0,POINT (-119.70650 34.42016),4341,93101,...,Y,0,0,0,0,0,0.0,0.0,0.0,0.0
4,93101.0,Santa Barbara,CA,34.420163,-119.7065,-8.0,1.0,POINT (-119.70650 34.42016),4342,93101,...,Y,0,0,0,0,0,0.0,0.0,0.0,0.0


In [54]:
pge_with_all_zip_coords_gpd[pge_with_all_zip_coords_gpd['zip'].isna()]['ZIPCODE'].unique()


array([93314, 93619, 93636, 93723, 93730, 93737, 94158, 94505, 94582,
       95391, 95467, 95757, 95811])

In [69]:
missing_zips_in_gpdf = {
    93314: {'city': 'Bakersfield',        'state': 'CA', 'latitude': 35.3821, 'longitude': -119.2088, 'timezone': -8.0, 'dst': 1.0 },
    93619: {'city': 'Clovis',             'state': 'CA', 'latitude': 36.9113, 'longitude': -119.5804, 'timezone': -8.0, 'dst': 1.0 },
    93636: {'city': 'Madera',             'state': 'CA', 'latitude': 36.9920, 'longitude': -119.8584, 'timezone': -8.0, 'dst': 1.0 },
    93723: {'city': 'Fresno',             'state': 'CA', 'latitude': 36.7703, 'longitude': -119.9509, 'timezone': -8.0, 'dst': 1.0 },
    93730: {'city': 'Fresno',             'state': 'CA', 'latitude': 36.9063, 'longitude': -119.7600, 'timezone': -8.0, 'dst': 1.0 },
    93737: {'city': 'Fresno',             'state': 'CA', 'latitude': 36.7513, 'longitude': -119.6441, 'timezone': -8.0, 'dst': 1.0 },
    94158: {'city': 'San Francisco',      'state': 'CA', 'latitude': 37.7748, 'longitude': -122.3873, 'timezone': -8.0, 'dst': 1.0 },
    94505: {'city': 'Discovery Bay',      'state': 'CA', 'latitude': 37.9012, 'longitude': -121.6022, 'timezone': -8.0, 'dst': 1.0 },
    94582: {'city': 'San Ramon',          'state': 'CA', 'latitude': 37.7639, 'longitude': -121.9132, 'timezone': -8.0, 'dst': 1.0 },
    95391: {'city': 'Tracy',              'state': 'CA', 'latitude': 37.7582, 'longitude': -121.5484, 'timezone': -8.0, 'dst': 1.0 },
    95467: {'city': 'Hidden Valley Lake', 'state': 'CA', 'latitude': 38.7977, 'longitude': -122.5429, 'timezone': -8.0, 'dst': 1.0 },
    95757: {'city': 'Elk Grove',          'state': 'CA', 'latitude': 38.3592, 'longitude': -121.4228, 'timezone': -8.0, 'dst': 1.0 },
    95811: {'city': 'Sacramnto',          'state': 'CA', 'latitude': 38.5967, 'longitude': -121.4942, 'timezone': -8.0, 'dst': 1.0 }
}

missing_zips_df=pd.DataFrame(missing_zips_in_gpdf).T.reset_index().rename({'index':'zip'}, axis=1)
missing_zips_gdf = gpd.GeoDataFrame(
    missing_zips_df, geometry=gpd.points_from_xy(missing_zips_df.longitude, missing_zips_df.latitude))
missing_zips_gdf.head()

,zip,city,state,latitude,longitude,timezone,dst,geometry
0,93314,Bakersfield,CA,35.3821,-119.209,-8,1,POINT (-119.20880 35.38210)
1,93619,Clovis,CA,36.9113,-119.58,-8,1,POINT (-119.58040 36.91130)
2,93636,Madera,CA,36.992,-119.858,-8,1,POINT (-119.85840 36.99200)
3,93723,Fresno,CA,36.7703,-119.951,-8,1,POINT (-119.95090 36.77030)
4,93730,Fresno,CA,36.9063,-119.76,-8,1,POINT (-119.76000 36.90630)


In [70]:
full_zips_gpdf = pd.concat([zips_gpd,missing_zips_gdf])
info(full_zips_gpdf)

(2729, 8)
Index(['zip', 'city', 'state', 'latitude', 'longitude', 'timezone', 'dst',
       'geometry'],
      dtype='object')


,zip,city,state,latitude,longitude,timezone,dst,geometry
0,95717,Gold Run,CA,39.177,-120.845,-8,1,POINT (-120.84510 39.17703)
1,94564,Pinole,CA,37.9975,-122.292,-8,1,POINT (-122.29208 37.99751)
2,91605,North Hollywood,CA,34.2081,-118.401,-8,1,POINT (-118.40110 34.20814)
3,91102,Pasadena,CA,33.7866,-118.299,-8,1,POINT (-118.29866 33.78659)
4,95019,Freedom,CA,36.9356,-121.78,-8,1,POINT (-121.77972 36.93555)


In [82]:
pge_with_all_zip_coords_gpd = pd.merge(left=full_zips_gpdf, right=full_cleaned_df, left_on="zip", right_on='ZIPCODE', how='inner').drop('Unnamed: 0',axis=1)
info(pge_with_all_zip_coords_gpd)


(154283, 24)
Index(['zip', 'city', 'state', 'latitude', 'longitude', 'timezone', 'dst',
       'geometry', 'ZIPCODE', 'DATE', 'MONTH', 'YEAR', 'DAY', 'CUSTOMERCLASS',
       'COMBINED', 'TOTALCUSTOMERS', 'TOTALKWH', 'AVERAGEKWH', 'TOTALTHERMS',
       'AVERAGETHERMS', 'Calculated AVGKWH', 'Calculated AVGTHERMS',
       'AVGKWH Diff', 'AVGTHERMS Diff'],
      dtype='object')


,zip,city,state,latitude,longitude,timezone,dst,geometry,ZIPCODE,DATE,...,COMBINED,TOTALCUSTOMERS,TOTALKWH,AVERAGEKWH,TOTALTHERMS,AVERAGETHERMS,Calculated AVGKWH,Calculated AVGTHERMS,AVGKWH Diff,AVGTHERMS Diff
0,95717,Gold Run,CA,39.177,-120.845,-8,1,POINT (-120.84510 39.17703),95717,2013-01-01,...,Y,0,0,0,0,0,0.00,0.0,0.00,0.0
1,95717,Gold Run,CA,39.177,-120.845,-8,1,POINT (-120.84510 39.17703),95717,2013-02-01,...,Y,0,0,0,0,0,0.00,0.0,0.00,0.0
2,95717,Gold Run,CA,39.177,-120.845,-8,1,POINT (-120.84510 39.17703),95717,2013-03-01,...,Y,0,0,0,0,0,0.00,0.0,0.00,0.0
3,95717,Gold Run,CA,39.177,-120.845,-8,1,POINT (-120.84510 39.17703),95717,2013-04-01,...,Y,0,0,0,0,0,0.00,0.0,0.00,0.0
4,95717,Gold Run,CA,39.177,-120.845,-8,1,POINT (-120.84510 39.17703),95717,2013-05-01,...,N,100,47657,477,0,0,476.57,0.0,-0.43,0.0


In [84]:
dest_fn = 'cleaned_data/full-pge-monthly-consumption_2013-2020_with-zipcodes-and-cities.shp'
pge_with_all_zip_coords_gpd.to_file(dest_fn)

# Comining with NREL Solar Radiation DB

reference: https://nsrdb.nrel.gov/data-sets/api-instructions.html  

In [80]:
full_nrel_csv_fn = 'cleaned_data/nrel_full.csv'
full_nrel_df = pd.read_csv(full_nrel_csv_fn)

In [81]:
info(full_nrel_df)

(14112, 12)
Index(['Unnamed: 0', 'Month', 'Year', 'GHI', 'DHI', 'DNI', 'Wind Speed',
       'Temperature', 'Solar Zenith Angle', 'zipcode', 'latitude',
       'longitude'],
      dtype='object')


,Unnamed: 0,Month,Year,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle,zipcode,latitude,longitude
0,0,1,2013.0,118.551075,30.461022,220.584677,3.564247,9.044355,105.478320,94002,37.516687,-122.29026
1,1,2,2013.0,166.961310,39.181548,269.005952,3.429167,9.815476,99.517798,94002,37.516687,-122.29026
2,2,3,2013.0,195.715054,71.255376,216.329301,3.340323,11.181452,91.103266,94002,37.516687,-122.29026
3,3,4,2013.0,284.418056,64.431944,334.995833,3.858194,12.713889,82.265542,94002,37.516687,-122.29026
4,4,5,2013.0,311.571237,71.837366,346.986559,4.352151,13.991935,75.565403,94002,37.516687,-122.29026
